### Assigment 4

**Submission deadlines**:

* get at least 4 points by Tuesday, 12.05.2022
* remaining points: last lab session before or on Tuesday, 19.05.2022

**Points:** Aim to get 12 out of 15+ possible points

All needed data files are on Drive: <https://drive.google.com/drive/folders/1HaMbhzaBxxNa_z_QJXSDCbv5VddmhVVZ?usp=sharing> (or will be soon :) )

## Task 1 (5 points)

Implement simplified word2vec with negative sampling from scratch (using pure numpy). Assume that in the training data objects and contexts are given explicitly, one pair per line, and objects are on the left. The result of the training should be object vectors. Please, write them to a file using *natural* text format, ie

<pre>
word1 x1_1 x1_2 ... x1_N 
word2 x2_1 x2_2 ... x2_N
...
wordK xK_1 xK_2 ... xk_N
</pre>

Use the loss from Slide 3 in Lecture NLP.2, compute the gradient manually. You can use some gradient clipping, or regularisation. 

**Remark**: the data is specially prepared to make the learning process easier. 
Present vectors using the code below. In this task we define success as 'obtaining a result which looks definitely not random'


In [1]:
from collections import defaultdict

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm

In [2]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

In [3]:
class Word2Vec:

    def __init__(self, data, dim, lr):

        self._data = data
        self._object_encoder = LabelEncoder()
        self._data['object'] = self._object_encoder.fit_transform(data['object'])
        self._context_encoder = LabelEncoder()
        self._data['context'] = self._context_encoder.fit_transform(data['context'])
        self._objects = np.unique(data[['object']].values)
        self._contexts = np.unique(data[['context']].values)
        self._vocab_count = None
        self._vocab_prob = None
        self._positives = None
        self._calculate_sampling_prob()
        self._prepare_positives()

        self._lr = lr
        self._dim = dim
        self._Wo = np.random.rand(len(self._objects), dim)
        self._Wc = np.random.rand(len(self._contexts), dim)

    def _calculate_sampling_prob(self):

        self._vocab_count = defaultdict(int)

        for word in self._data['object']:
            self._vocab_count[word] += 1

        norm = sum([freq**(3/4) for freq in self._vocab_count.values()])
        self._vocab_prob = {word: (freq** (3 / 4) / norm) for word, freq in self._vocab_count.items()}

    def _prepare_positives(self):

        self._positives = defaultdict(set)
        for _, row in tqdm(self._data.iterrows()):
            self._positives[row.context] |= {row.object}

    def update_object_embedding(self, word_idx, gradient):

        self._Wo[word_idx, :] -= self._lr * gradient

    def update_context_embedding(self, word_idx, gradient):

        self._Wc[word_idx, :] -= self._lr * gradient

    def get_negatives_sample(self, k):

        negatives = np.random.choice(list(self._vocab_prob.keys()), size=(len(self._data), k), p=list(self._vocab_prob.values()))

        return negatives

    def step(self, idx, negatives):
        word_idx, context_idx = self._data.iloc[idx]
        negatives = list(set(negatives) - set(self._positives[context_idx]))

        loss = -np.log(sigmoid(self._Wo[word_idx, :] @ self._Wc[context_idx, :])) - np.sum(np.log(sigmoid(-self._Wo[negatives, :] @ self._Wc[context_idx, :])))
        derivative_object = -self._Wc[context_idx, :]*(1 - sigmoid(self._Wo[word_idx, :] @ self._Wc[context_idx, :]))
        derivative_context = -self._Wo[word_idx, :]*(1 - sigmoid(self._Wo[word_idx, :] @ self._Wc[context_idx, :])) \
                             + np.sum(self._Wo[negatives, :].T * (1 - sigmoid(-self._Wc[context_idx, :].reshape(1, -1) @ self._Wo[negatives, :].T)), axis=1)

        self.update_object_embedding(word_idx, derivative_object)
        self.update_context_embedding(context_idx, derivative_context)

        return loss

    def save_embeddings(self, path):
        with open(path, 'w') as f:
            f.write(f'{str(len(self._objects))} {str(self._dim)} \n')
            for word in self._objects:
                embedding = self._Wo[word]
                f.write(f'{self._object_encoder.inverse_transform([word])[0]} {" ".join(list(map(str, list(embedding))))}  \n')

In [4]:
data = pd.read_csv('task1_objects_contexts_polish.txt', sep=' ', names=['object', 'context'])

In [5]:
w = Word2Vec(data, 20, 0.01)

0it [00:00, ?it/s]

In [6]:
for epoch in range(20):
    negatives = w.get_negatives_sample(5)
    print(f"EPOCH {epoch}")
    losses = []
    for i in tqdm(range(len(negatives))):
        loss = w.step(i, negatives[i])
        losses.append(loss)
    print(np.mean(np.array(losses)))

EPOCH 0


  0%|          | 0/5525116 [00:00<?, ?it/s]

6.07505048998839
EPOCH 1


  0%|          | 0/5525116 [00:00<?, ?it/s]

4.205103797833125
EPOCH 2


...

3.961706487544347
EPOCH 18


  0%|          | 0/5525116 [00:00<?, ?it/s]

3.963314780543266
EPOCH 19


  0%|          | 0/5525116 [00:00<?, ?it/s]

3.9665752325498493


In [7]:
w.save_embeddings('task1_w2v_vectors.txt')

In [11]:
from gensim.models import KeyedVectors
task1_wv = KeyedVectors.load_word2vec_format('task1_w2v_vectors.txt', binary=False)

example_english_words = ['dog', 'dragon', 'love', 'bicycle', 'marathon', 'logic', 'butterfly']  # replace, or add your own examples
example_polish_words = ['pies', 'smok', 'miłość', 'rower', 'maraton', 'logika', 'motyl']

example_words = example_polish_words

for w0 in example_words:
    print ('WORD:', w0)
    for w, v in task1_wv.most_similar(w0):
        print ('   ', w, v)
    print ()

WORD: pies
    koń 0.9517124891281128
    kot 0.9100876450538635
    krowa 0.9069613814353943
    zwierzę 0.8956605195999146
    baba 0.8795665502548218
    dziewczyna 0.8694310784339905
    mężczyzna 0.8668525815010071
    ptak 0.8655155301094055
    chłopiec 0.8645903468132019
    facet 0.8518747687339783

WORD: smok
    potwór 0.8994752764701843
    niedźwiedź 0.896720826625824
    dziewczę 0.896420419216156
    kot 0.8909311890602112
    maluch 0.8871726989746094
    panienka 0.8804367780685425
    dzieciak 0.877396285533905
    mucha 0.8752036690711975
    wiedźma 0.8732087016105652
    tygrys 0.869754433631897

WORD: miłość
    wiara 0.8628854155540466
    duch 0.7667196393013
    prawda 0.766288161277771
    bóg 0.760909914970398
    wyobraźnia 0.7606624364852905
    młodość 0.7587139010429382
    radość 0.7577516436576843
    zło 0.7552950382232666
    uczucie 0.7470588088035583
    przyjaźń 0.7453605532646179

WORD: rower
    motocykl 0.9155734777450562
    wózek 0.90378886461

## Task 2 (4 points)

Your task is to train the embeddings for Simple Wikipedia titles, using gensim library. As the example below shows, training is really simple:

```python
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
model = Word2Vec(sentences=common_texts, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")
```
*sentences* can be a list of list of tokens, you can also use *gensim.models.word2vec.LineSentence(source)* to create restartable iterator from file. At first, use [this file] containing such pairs of titles, that one article links to another.

We say that two titles are *related* if they both contain a word (or a word bigram) which is not very popular (it occurs only in several titles). Make this definition more precise, and create the corpora which contains pairs of related titles. Make a mixture of the original corpora, and the new one, then train title vectors again.

Compare these two approaches using similar code to the code from Task 1.

In [3]:
# The cell for your presentation
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence


# model = Word2Vec(sentences=common_texts, vector_size=100, window=5, min_count=1, workers=4)
# model.save("word2vec.model")

In [4]:
data = LineSentence('simple_wiki_links.txt')

In [6]:
model = Word2Vec(sentences=data, vector_size=100, window=5, min_count=1, workers=4)

In [11]:
example_titles = ["hyperinflation", "kevin_spacey", "animal", "catholicism"]
for word in example_titles:
    print(f'WORD: {word}')
    for similar_word, score in model.wv.most_similar(word, topn=5):
        print(similar_word)
    print()

wartburgkreis
rhein-sieg-kreis
ministry_of_labour_and_social_affairs_of_germany
alb-donau-kreis
wolfgang_thonke
jack_nicholson
garry_shandling
ben_affleck
robert_duvall
jerry_seinfeld
mammal
species
plant
genus
bird
crusades
yuri_gagarin
colonialism
holocaust_victims
ustaše


# Task 3 (4 points)

Suppose that we have two languages: Upper and Lower. This is an example Upper sentence:

<pre>
THE QUICK BROWN FOX JUMPS OVER THE LAZY DOG.
</pre>

And this is its translation into Lower:

<pre>
the quick brown fox jumps over the lazy dog
</pre>

You have two corpora for these languages (with different sentences). Your task is to train word embedings for both languages together, so as to make embeddings of the words which are its translations as close as possible. But unfortunately, you have the budget which allows you to prepare the translation only for 1000 words (we call it D, you have to deside which words you want to be in D)

Prepare the corpora wich contains three kind of sentences:
* Upper corpus sentences
* Lower corpus sentences
* sentences derived from Upper/Lower corpus, modified using D

There are many possible ways of doing this, for instance this one (ROT13.COM: hfr rirel fragrapr sebz obgu pbecben gjvpr: jvgubhg nal zbqvsvpngvbaf, naq jvgu rirel jbeqf sebz Q ercynprq ol vgf genafyngvba)

We define the score for an Upper WORD as  $\frac{1}{p}$, where $p$ is a position of its translation in the list of **Lower** words most similar to WORD. For instance, when most similar words to DOG are:

<pre>
WOLF, CAT, WOLVES, LION, gopher, dog
</pre>

then the score for the word DOG is 0.5. Compute the average score separately for words from D, and for words out of D (hint: if the computation takes to much time do it for a random sample).


# Task 4 (4 points)

In this task you are asked to do two things:
1. compare the embeddings computed on small corpus (like Brown Corpus , see: <https://en.wikipedia.org/wiki/Brown_Corpus>) with the ones coming from Google News Corpus
2. Try to use other resourses like WordNet to enrich to corpus, and obtain better embeddings

You can use the following code snippets:

```python
# printing tokenized Brown Corpora
from nltk.corpus import brown
for s in brown.sents():
    print(*s)
    
#iterating over all synsets in WordNet
from nltk.corpus import wordnet as wn

for synset_type in 'avrns': # n == noun, v == verb, ...
    for synset in list(wn.all_synsets(synset_type)))[:10]:
        print (synset.definition())
        print (synset.examples())
        print ([lem.name() for lem in synset.lemmas()])
        print (synset.hyperonims()) # nodes 1 level up in ontology
        
# loading model and compute cosine similarity between words

model = Word2Vec.load('models/w2v.wordnet5.model') 
print (model.wv.similarity('dog', 'cat'))
```

Embeddings will be tested using WordSim-353 dataset, the code showing the quality is in the cell below. Prepare the following corpora:
1. Tokenized Brown Corpora
2. Definitions and examples from Princeton WordNet
3. (1) and (2) together
4. (3) enriched with pseudosentences containing (a subset) of WordNet knowledge (such as 'tiger is a carnivore')

Train 4 Word2Vec models, and raport Spearman correletion between similarities based on your vectors, and similarities based on human judgements.



In [10]:
# Code for computing correlation between W2V similarity, and human judgements

import gensim.downloader
from scipy.stats import spearmanr

gn = gensim.downloader.load('word2vec-google-news-300')

for similarity_type in ['relatedness', 'similarity']:
    ws353 = []
    for x in open(f'wordsim_{similarity_type}_goldstandard.txt'): 
        a,b,val = x.split()
        val = float(val)
        ws353.append( (a,b,val))
    # spearmanr returns 2 vallues: correlation and pval. pval should be close to zero
    print (similarity_type + ':', spearmanr(vals, ys)) 

[==================================================] 100.0% 1662.8/1662.8MB downloaded


FileNotFoundError: [Errno 2] No such file or directory: 'wordsim_relatedness_goldstandard.txt'